# cadCAD Experiment
Over 500 timesteps, money comes in to the system each timestep and is distributed to delegators according to the proportion of shares they have bought.  The share price and number are tied to a bonding curve.  

They purchase shares in the system according to their belief of the future revenue streams.

In [ ]:
# if this crashes, run this:
# pip install ipython-autotime
%load_ext autotime
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")





In [ ]:
import matplotlib.pyplot as plt
from model.run import run

# df = run()


In [ ]:
from cadCAD_tools import profile_run

from model.psub import psubs
from model.state import genesis_state
from model.config import simulation_config
from model.config import params

TIMESTEPS = len(simulation_config[0]['T'])
SAMPLES = simulation_config[0]['N'] 
# df = run()
df = profile_run(genesis_state,
                 params,
                 psubs,
                 TIMESTEPS,
                 SAMPLES)

from cadCAD_tools.profiling.visualizations import visualize_substep_impact

visualize_substep_impact(df, relative=True)


In [ ]:
df

# Revenue coming in each period.  
## Mean is a parameter, currently 7
## It's exponentially distributed.

In [ ]:
df.period_revenue.plot()

# Cumulative Claimed Rewards
This shows that the cumulative rewards continues to rise even though each delegator eventually leaves the system and their rewards become fixed.  They are of similar heights (ie. same rewards) because they all use the same policies to join and leave.

In [ ]:
import pandas as pd
df1 = df.reset_index().delegators

delegator_revenue_token_holdings = {}
delegator_shares = {}
timestep = 0
for delegator_dict in df1:
    for delegator_id, delegator in delegator_dict.items():
        if delegator_id not in delegator_revenue_token_holdings:
            delegator_revenue_token_holdings[delegator_id] = {}
            delegator_shares[delegator_id] = {}
        delegator_revenue_token_holdings[delegator_id][timestep] = delegator.revenue_token_holdings
        delegator_shares[delegator_id][timestep] = delegator.shares
        
    timestep += 1

# Shares broken down by Delegator

In [ ]:


df_plot = pd.DataFrame(delegator_shares)
df_plot.plot(kind='area')
plt.gca().legend(loc=9, bbox_to_anchor=(2.5, 1), ncol=12)
plt.title('Shares Owned By Delegators')
plt.xlabel('Epoch Number')
plt.ylabel('Delegator Shares')

# Cumulative Revenue Token Holdings



In [ ]:
df_plot = pd.DataFrame(delegator_revenue_token_holdings)
df_plot.plot(kind='area')
plt.gca().legend(loc=9, bbox_to_anchor=(2.5, 1), ncol=12)
plt.title('Cumulative Revenue Token Holdings')
plt.xlabel('Epoch Number')
plt.ylabel('Revenue Token Holdings')

Exploratory data analysis

In [ ]:
df['shareholders'] = df.delegators.apply(lambda x: [k for k in x.keys() if x[k].shares>0] )

In [ ]:
df['shares'] = df.delegators.apply(lambda x: [x[k].shares for k in x.keys() if x[k].shares>0] )

In [ ]:
# cannot currently do this but i want to for plotting purposes
# df['private_prices'] = df.delegators.apply(lambda x: [x[k].private_price for k in x.keys()] )
df['private_prices'] = df.delegators.apply(lambda x: {k: x[k].private_price for k in x.keys()} )

In [ ]:
df_list = []
for t in range(len(df)):
    temp_df = pd.DataFrame({'shareholder':df.shareholders.iloc[t] , 'shares':df.shares.iloc[t]})
    temp_df['Epoch Number'] = t

    df_list.append(temp_df)

shareholder_df = pd.concat(df_list)

In [ ]:
shareholder_df

# Price per share

In [ ]:
df.spot_price.plot()

In [ ]:
df['period_revenue_per_share'] = df.period_revenue/df.supply

# Revenue per share 

In [ ]:
df.period_revenue_per_share.plot()

# Estimated Share Valuation

In [ ]:
#estimated share valuation
risk_adj = .7 #if agents get their own param then there exists an estimated value of this system level param
discount_factor = .95 #if agents get their own param there exist an estimated value of this systel level param

df['share_valuation'] = df.spot_price * risk_adj + df.period_revenue_per_share / (1 - discount_factor)
df['ewm_share_valuation'] = df.share_valuation.ewm(halflife = 10).mean()

In [ ]:
df[['share_valuation', 'spot_price', 'ewm_share_valuation', 'timestep']].plot(x='timestep', color='ybg')

In [ ]:
df.columns

In [ ]:
import matplotlib.pylab as plt
df[1:7501:15]
x = range(500)
# for t in range(1,7501,15):
d = df['private_prices']
timestep = 0
delegateId = 0
x1 = []
y1 = []
fix, ax = plt.subplots()

for timestep in x:
    for delegateId in range(25):    
        # The data has to be in the form x = [timesteps], y = [values]
        # the data is in the form y = dict({key=delegator, value=})
        if delegateId in d[timestep]:
            x1.append(timestep)
            y1.append(d[timestep][delegateId])
    plt.plot(x1, y1)

        # print(d[timestep][delegateId])
        # y = [y for y in d]
    # d = df['private_prices'][i]

# print(d[40])
# print(df['private_prices'])
# lists = sorted(d.items())
# print(lists)
# x, y = zip(*lists) # unpack a list of pairs into two tuples
# print(y)
# # plt.plot(x, y)
plt.title('Private Price Estimates for first 25 Delegators')
plt.ylabel('Delegator Price Estimate')
plt.xlabel('Timestep')

# plt.show()